In [ ]:
pip install flask dash dash-core-components dash-html-components

In [ ]:
from flask import Flask
import dash
import dash_core_components as dcc
import dash_html_components as html
from sklearn.datasets import load_iris
import plotly.express as px
import pandas as pd

# Load the Iris dataset from the standard library
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = iris.target_names[iris.target]

# Create the Flask app
server = Flask(__name__)
app = dash.Dash(__name__, server=server)

# Define available species options for dropdown
species_options = [{'label': species, 'value': species} for species in df['species'].unique()]

# Define available feature options for dropdown
feature_options = [{'label': feature, 'value': feature} for feature in df.columns[:-1]]

# Define the layout of the dashboard
app.layout = html.Div(
    style={'margin': '25px'},
    children=[
        html.H1(children='Iris Dataset Dashboard'),
        html.Div(
            className='about-section',
            children=[
                html.H2('About the Iris Dataset'),
                html.P('The Iris dataset consists of measurements of four features (sepal length, sepal width, petal length, petal width) for three different species of Iris flowers (Setosa, Versicolor, and Virginica).'),
                html.P('Use the dropdown menus below to select a species and the x and y axes for each plot.'),
            ]
        ),
        html.Div(
            className='species-dropdown-section',
            children=[
                html.Label('Select Species:'),
                dcc.Dropdown(
                    id='species-dropdown',
                    options=species_options,
                    value=df['species'].unique()[0]
                )
            ]
        ),
        html.Div(
            className='plots-section',
            children=[
                html.Div(
                    className='plot',
                    children=[
                        html.H3('Scatter Plot'),
                        html.Label('X-axis:'),
                        dcc.Dropdown(
                            id='scatter-plot-x',
                            options=feature_options,
                            value=df.columns[0]
                        ),
                        html.Label('Y-axis:'),
                        dcc.Dropdown(
                            id='scatter-plot-y',
                            options=feature_options,
                            value=df.columns[1]
                        ),
                        dcc.Graph(id='scatter-plot')
                    ]
                ),
                html.Div(
                    className='plot',
                    children=[
                        html.H3('Histogram'),
                        html.Label('X-axis:'),
                        dcc.Dropdown(
                            id='histogram-x',
                            options=feature_options,
                            value=df.columns[0]
                        ),
                        dcc.Graph(id='histogram')
                    ]
                ),
                html.Div(
                    className='plot',
                    children=[
                        html.H3('Box Plot'),
                        html.Label('Y-axis:'),
                        dcc.Dropdown(
                            id='box-plot-y',
                            options=feature_options,
                            value=df.columns[4]
                        ),
                        dcc.Graph(id='box-plot')
                    ]
                ),
                html.Div(
                    className='plot',
                    children=[
                        html.H3('Violin Plot'),
                        html.Label('Y-axis:'),
                        dcc.Dropdown(
                            id='violin-plot-y',
                            options=feature_options,
                            value=df.columns[4]
                        ),
                        dcc.Graph(id='violin-plot')
                    ]
                )
            ]
        )
    ]
)

# Define callback for updating the scatter plot based on selected species and axes
@app.callback(
    dash.dependencies.Output('scatter-plot', 'figure'),
    [dash.dependencies.Input('species-dropdown', 'value'),
     dash.dependencies.Input('scatter-plot-x', 'value'),
     dash.dependencies.Input('scatter-plot-y', 'value')]
)
def update_scatter_plot(species, x_axis, y_axis):
    filtered_df = df[df['species'] == species]
    fig = px.scatter(filtered_df, x=x_axis, y=y_axis, color='species')
    fig.update_layout(transition_duration=500)
    return fig

# Define callback for updating the histogram based on selected species and axis
@app.callback(
    dash.dependencies.Output('histogram', 'figure'),
    [dash.dependencies.Input('species-dropdown', 'value'),
     dash.dependencies.Input('histogram-x', 'value')]
)
def update_histogram(species, x_axis):
    filtered_df = df[df['species'] == species]
    fig = px.histogram(filtered_df, x=x_axis, nbins=20)
    fig.update_layout(transition_duration=500)
    return fig

# Define callback for updating the box plot based on selected species and axis
@app.callback(
    dash.dependencies.Output('box-plot', 'figure'),
    [dash.dependencies.Input('species-dropdown', 'value'),
     dash.dependencies.Input('box-plot-y', 'value')]
)
def update_box_plot(species, y_axis):
    filtered_df = df[df['species'] == species]
    fig = px.box(filtered_df, y=y_axis, color='species')
    fig.update_layout(transition_duration=500)
    fig.update_xaxes(showticklabels=False)  # Remove x-axis tick labels
    return fig


# Define callback for updating the violin plot based on selected species and axis
@app.callback(
    dash.dependencies.Output('violin-plot', 'figure'),
    [dash.dependencies.Input('species-dropdown', 'value'),
     dash.dependencies.Input('violin-plot-y', 'value')]
)
def update_violin_plot(species, y_axis):
    filtered_df = df[df['species'] == species]
    fig = px.violin(filtered_df, y=y_axis, box=True, points='all', color='species')
    fig.update_layout(transition_duration=500)
    return fig

# Run the Flask app
if __name__ == '__main__':
    app.run_server(debug=False)

